# Hydrogenation Case

<div><img src="./cases/hydrogenation_process_scheme.jpg" width="600"/></div>

This case aims to model a Hydrogenation process from R-di-yne to the desired R-di-ethyl. During the process, 
RDY and H2 are dissolved in to the mixed liquid of water and a unknown solvent. Agitation is applied to enhance 
mixing. A cooling jacket is set to remove heat produced by exothermal hydrogenation reactions.  
The Di-yne group of RDY is gradually hydrogenated by cat-H through a series of hydrogenation reactions, 
from RDY, to RYE, RYA, REA, and finally RDEt. Side reactions generating dimer can occur in the process.  

The reaction scheme is like:
<div><img src="./cases/hydrogenation_reaction_scheme.png" width="900"/></div>

Reactions:
- H2 + cat2 > 2 cat-H
- 2 cat-H > H2 + cat2
- RDY + 2 cat-H > RYE + cat2
- RYE + 2 cat-H > RYA + cat2
- RYA + 2 cat-H > REA + cat2
- REA + 2 cat-H > RDEt + cat2
- RDEt + catX > catX-RDEt
- catX-RDEt > RDEt + catX
- RYE + catX-RDEt > dimer + catX
- REA + catX-RDEt > 2 H2 + dimer + catX

In this notebook, we go through model simulation and identify the rate-limiting step of the hydrogenation reaction.  
Then, sensitivity analysis is applied to identify parameters to calibrated.  
Finally, we run model calibration with different mass transport behaviours for model identification.

In [1]:
# import required python libraries
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from bayes_opt import BayesianOptimization, acquisition
from plotly.offline import init_notebook_mode

from cases.hydrogenation import Hydrogenation

init_notebook_mode(connected=True)

# define phenomenon for the process to define the model
# RDY saturated concentration is fitted with RDEt concentration
# supported phenomenon options
# "Mass transport":
#       []
#       ["Gas-Liquid_Mass_Transfer"]
#       ["Solid-Liquid_Mass_Transfer"]
#       ["Gas-Liquid_Mass_Transfer", "Solid-Liquid_Mass_Transfer"],
# is "Gas-Liquid_Mass_Transfer" occurs, "Gas_Dissolution_Saturated_Concentration" can be "Henry's_Law" or "Constant" 
# is "Solid-Liquid_Mass_Transfer" occurs, "Solid_Dissolution_Saturated_Concentration" can be "Fitted" or "Constant" 
# RDY dissolution saturated concentration is found dependent on RDEt concentration, a relationship has been fitted and 
# included in the `hydrogenation.py` file for modelling
phenos = {
    "Mass accumulation":    "Batch",
    "Flow pattern":         "Well_Mixed",
    "Mass transport":       ["Gas-Liquid_Mass_Transfer", "Solid-Liquid_Mass_Transfer"],
    "param law":            {
        "Gas_Dissolution_Saturated_Concentration": "Henry's_Law",
        "Solid_Dissolution_Saturated_Concentration": "Fitted", 
    },
}

hydrogenation = Hydrogenation(phenos, random_seed=42)

### Model Simulation and Rate-limiting Step analysis
In this section, we run hydrogenation simulation with guessed kinetic parameters to visualise concentration profiles of reactants, intermediates, and products.  
Rate-limiting step is analysed by plotting intermediate mass profiles and rates of reaction and mass transport, which is further confirmed by sensitivity analysis.

In [2]:
# list of operation parameters
# During experiments, masses of used water, solvent, and catalysts are fixed.
hydrogenation.operation_params()

{('Initial_Mass', None, 'batch_stream', None, 'water'): 0.2574,
 ('Initial_Mass', None, 'batch_stream', None, 'solvent'): 0.194,
 ('Initial_Mass', None, 'batch_stream', None, 'cat2'): 0.00235,
 ('Initial_Mass', None, 'batch_stream', None, 'catX'): 0.00235,
 ('Mass_Gas_Fraction', 'gas_flow', None, None, 'H2'): 1.0,
 ('Initial_Mass_Solid', 'solid_feedstock', None, None, 'RDY'): None,
 ('Temperature', None, None, None, None): None,
 ('Pressure', None, None, None, None): None,
 ('Batch_Time', None, None, None, None): None}

In [3]:
# model simulation and plot concentration profiles of RDY, RYE, RYA, REA, RDEt, dimer
operation_params = {
    ('Initial_Mass_Solid', 'solid_feedstock', None, None, 'RDY'):   0.08784,
    ('Temperature', None, None, None, None):                        20,
    ('Pressure', None, None, None, None):                           10,
    ('Batch_Time', None, None, None, None):                         152,
}
kinetics_params = hydrogenation.kinetics_params()
hydrogenation.plot_simulation_profiles(operation_params)

In [4]:
# plot all reaction and mass transfer rates
# it can be observed that rate of "H2 dissolution" and "H2 + cat2 <> 2 cat-H" are relatively fast
operation_params = {
    ('Initial_Mass_Solid', 'solid_feedstock', None, None, 'RDY'):   0.08784,
    ('Temperature', None, None, None, None):                        20,
    ('Pressure', None, None, None, None):                           10,
    ('Batch_Time', None, None, None, None):                         152,
}
steps = [
    "H2 + cat2 > 2 cat-H",
    "2 cat-H > H2 + cat2",
    "RDY + 2 cat-H > RYE + cat2",
    "RYE + 2 cat-H > RYA + cat2",
    "RYA + 2 cat-H > REA + cat2",
    "REA + 2 cat-H > RDEt + cat2",
    "RDEt + catX > catX-RDEt",
    "catX-RDEt > RDEt + catX",
    "RYE + catX-RDEt > dimer + catX",
    "REA + catX-RDEt > 2 H2 + dimer + catX",
    "H2 dissolution", 
    "RDY dissolution"
]
hydrogenation.plot_simulation_rates(steps, operation_params)

In [5]:
# plot only rates of reactions from RDY to RDEt and RDY dissolution rate
# note that there's rate-limiting step in the series of hydrogenation reaction based on the overlapped rate curves
# RDY and RYE accumulate rapidly and then decay slowly
# if "RDY dissolution" is the rate-limiting step, there will be no RDY and RYA mass accumulation
# if "RYA + 2 cat-H > REA + cat2" is the rate-limiting step, RYA will accumulate which is not the case
# Therefore, "RDY + 2 cat-H > RYE + cat2" is the rate-limiting step, and RDY concentration rises close
# to the saturated concentration to slow down the solid RDY consumption rate
operation_params = {
    ('Initial_Mass_Solid', 'solid_feedstock', None, None, 'RDY'):   0.08784,
    ('Temperature', None, None, None, None):                        20,
    ('Pressure', None, None, None, None):                           10,
    ('Batch_Time', None, None, None, None):                         152,
}
steps = [
    "RDY + 2 cat-H > RYE + cat2",
    "RYE + 2 cat-H > RYA + cat2",
    "RYA + 2 cat-H > REA + cat2",
    "REA + 2 cat-H > RDEt + cat2",
    "RDY dissolution"
]
hydrogenation.plot_simulation_rates(steps, operation_params)

In [6]:
# plot RDEt mass profiles under varied mechanistic parameters
operation_params = {
    ('Initial_Mass_Solid', 'solid_feedstock', None, None, 'RDY'):   0.08784,
    ('Temperature', None, None, None, None):                        20,
    ('Pressure', None, None, None, None):                           10,
    ('Batch_Time', None, None, None, None):                         152,
}
varied_mechanistic_params = {
    "original": None,
    "H2 dissolution": ("Gas-Liquid_Volumetric_Mass_Transfer_Coefficient", "gas_flow", "batch_stream", None, "H2"),
    "RDY dissolution": ("Solid-Liquid_Volumetric_Mass_Transfer_Coefficient", "solid_feedstock", "batch_stream", None, "RDY"),
    "RDY hydrogenation": ("Referenced_Reaction_Rate_Constant", None, "batch_stream", "RDY + 2 cat-H > RYE + cat2", None),
    "RYA hydrogenation": ("Referenced_Reaction_Rate_Constant", None, "batch_stream", "RYA + 2 cat-H > REA + cat2", None),
    "Dimer generation1": ("Referenced_Reaction_Rate_Constant", None, "batch_stream", "RYE + catX-RDEt > dimer + catX", None),
    "Dimer generation2": ("Referenced_Reaction_Rate_Constant", None, "batch_stream", "REA + catX-RDEt > 2 H2 + dimer + catX", None),
}
hydrogenation.plot_sensitivity_analysis(varied_mechanistic_params, operation_params)

### Model calibration and identification
We first compare prediction vs. experiment data using parameters without calibration

In [7]:
# load experiment data and print
exp_dataset = pd.read_csv("./cases/hydrogenation_dataset.csv")
exp_dataset

,m_rdy,t_b,temp,pressure,m_rdet,m_dimer,h2_used,m_rdy_left
0,0.10444,311,10,4,0.0662,0.00244,0.00339,0.03330
1,0.10444,309,20,4,0.0838,0.00663,0.00445,0.01149
2,0.09060,135,25,12,0.0862,0.00362,0.00454,0.00252
3,0.08520,352,25,4,0.0739,0.00813,0.00410,0.00209
4,0.08770,352,5,12,0.0766,0.00133,0.00381,0.01380


In [8]:
# plot parity plots
hydrogenation.plot_simulation_parity(exp_dataset)

#### Determining parameters for calibration
Recall reactions and mass transport phenomena
- H2 + cat2 > 2 cat-H
- 2 cat-H > H2 + cat2
- RDY + 2 cat-H > RYE + cat2
- RYE + 2 cat-H > RYA + cat2
- RYA + 2 cat-H > REA + cat2
- REA + 2 cat-H > RDEt + cat2
- RDEt + catX > catX-RDEt
- catX-RDEt > RDEt + catX
- RYE + catX-RDEt > dimer + catX
- REA + catX-RDEt > 2 H2 + dimer + catX
- hydrogen dissolution
- RDY dissolution

Measurements
- RDEt
- Dimer
- RDY undissolved
- H2 used

Parameters to be calibrated and their pristine values
- RDY hydrogenation reaction [RDY + 2 cat-H > RYE + cat2]  
    Referenced_Reaction_Rate_Constant: 3 mol/L s  
    Activation_Energy: 30 kJ/mol
- Dimer generation reaction [RYE + catX-RDEt > dimer + catX]  
    Referenced_Reaction_Rate_Constant: 1 mol/L s  
    Activation_Energy: 38 kJ/mol
- Dimer generation reaction [REA + catX-RDEt > 2 H2 + dimer + catX]  
    Referenced_Reaction_Rate_Constant: 1 mol/L s  
    Activation_Energy: 38 kJ/mol

In [9]:
# run model calibration with differential_evolution algorithm for complex reaction network model
cal_param_ranges = {
    ("Referenced_Reaction_Rate_Constant", None, "batch_stream", "RDY + 2 cat-H > RYE + cat2", None): (0, 10),
    ("Referenced_Reaction_Rate_Constant", None, "batch_stream", "RYE + catX-RDEt > dimer + catX", None): (0, 10),
    ("Referenced_Reaction_Rate_Constant", None, "batch_stream", "REA + catX-RDEt > 2 H2 + dimer + catX", None):(0, 10),
    ("Activation_Energy", None, "batch_stream", "RDY + 2 cat-H > RYE + cat2", None): (20, 80),
    ("Activation_Energy", None, "batch_stream", "RYE + catX-RDEt > dimer + catX", None): (20, 80),
    ("Activation_Energy", None, "batch_stream", "REA + catX-RDEt > 2 H2 + dimer + catX", None): (20, 80),
    ("Solid-Liquid_Volumetric_Mass_Transfer_Coefficient", "solid_feedstock", "batch_stream", None, "RDY"): (10, 40),
}
cal_params = hydrogenation.calibrate(cal_param_ranges, exp_dataset)

differential_evolution step 1: f(x)= 0.0006546489453876825
differential_evolution step 2: f(x)= 0.000291475685448116
differential_evolution step 3: f(x)= 0.000291475685448116
differential_evolution step 4: f(x)= 0.000291475685448116
differential_evolution step 5: f(x)= 0.000291475685448116
differential_evolution step 6: f(x)= 0.000291475685448116
differential_evolution step 7: f(x)= 0.000291475685448116
differential_evolution step 8: f(x)= 0.000291475685448116
differential_evolution step 9: f(x)= 0.00027912489636440176
differential_evolution step 10: f(x)= 0.0002711934593916121


In [10]:
# parity plot after model calibratioin
cal_kinetics_params = {k: v for k, v in cal_params.items() if k in hydrogenation.kinetics_params()}
cal_transport_params = {k: v for k, v in cal_params.items() if k in hydrogenation.transport_params()}
hydrogenation.plot_simulation_parity(exp_dataset, kinetics_params=cal_kinetics_params, transport_params=cal_transport_params)

#### Fitting model with constant RDY saturated concentration
identify how should we model the RDY saturated concentration by comparing calibration results of different models

In [11]:
# try to identify the best fitted model with different structures based on different phenomenons
phenos = {
    "Mass accumulation":    "Batch",
    "Flow pattern":         "Well_Mixed",
    "Mass transport":       ["Gas-Liquid_Mass_Transfer", "Solid-Liquid_Mass_Transfer"],
    "param law":            {
        "Gas_Dissolution_Saturated_Concentration": "Henry's_Law",
        "Solid_Dissolution_Saturated_Concentration": "Constant",
    },
}

hydrogenation = Hydrogenation(phenos, random_seed=42)

In [12]:
# run model calibration again with constant solid saturation concentration
cal_param_ranges = {
    ("Referenced_Reaction_Rate_Constant", None, "batch_stream", "RDY + 2 cat-H > RYE + cat2", None): (0, 10),
    ("Referenced_Reaction_Rate_Constant", None, "batch_stream", "RYE + catX-RDEt > dimer + catX", None): (0, 10),
    ("Referenced_Reaction_Rate_Constant", None, "batch_stream", "REA + catX-RDEt > 2 H2 + dimer + catX", None):(0, 10),
    ("Activation_Energy", None, "batch_stream", "RDY + 2 cat-H > RYE + cat2", None): (20, 100),
    ("Activation_Energy", None, "batch_stream", "RYE + catX-RDEt > dimer + catX", None): (20, 100),
    ("Activation_Energy", None, "batch_stream", "REA + catX-RDEt > 2 H2 + dimer + catX", None): (20, 100),
    ("Constant_Solid_Saturated_Concentration", "solid_feedstock", "batch_stream", None, "RDY"): (0, 0.003),
}
cal_params = hydrogenation.calibrate(cal_param_ranges, exp_dataset)

differential_evolution step 1: f(x)= 0.0007585428161100219
differential_evolution step 2: f(x)= 0.0007421944979813215
differential_evolution step 3: f(x)= 0.0005938039342365542
differential_evolution step 4: f(x)= 0.0005938039342365542
differential_evolution step 5: f(x)= 0.0005938039342365542
differential_evolution step 6: f(x)= 0.0005938039342365542
differential_evolution step 7: f(x)= 0.0005938039342365542
differential_evolution step 8: f(x)= 0.0005938039342365542
differential_evolution step 9: f(x)= 0.0005925662282789956
differential_evolution step 10: f(x)= 0.0005925662282789956


In [13]:
# again, plot parity results and find that model with constant solid saturation concentration is inferior
# this suggests that RDY dissolution is more reasonable to be related to the concentration of RDEt
cal_kinetics_params = {k: v for k, v in cal_params.items() if k in hydrogenation.kinetics_params()}
cal_transport_params = {k: v for k, v in cal_params.items() if k in hydrogenation.transport_params()}
hydrogenation.plot_simulation_parity(exp_dataset, kinetics_params=cal_kinetics_params, transport_params=cal_transport_params)